In [1]:
# Get Pest Data:

In [2]:
import pandas as pd
import requests
import time
from io import StringIO

In [3]:
# Configuration 
import configparser

# Create ConfigParser object
config = configparser.ConfigParser()
config.read("Config.ini")

API_URL = config.get("1_PEST", "API_URL")
USERNAME = config.get("1_PEST", "USERNAME")
PASSWORD = config.get("1_PEST", "PASSWORD")
fran = config.get("1_PEST", "fran")
till = config.get("1_PEST", "till")
groda = config.get("1_PEST", "groda")

columns_to_select = config.get("1_PEST", "columns_to_select").replace("\n", "").split(", ")
n_pest_coordinates = config.getint("1_PEST", "n_pest_coordinates")

pkl_filename = config.get("1_PEST", "pkl_filename")
csv_filename = config.get("1_PEST", "csv_filename")
pkl_all_filename = config.get("1_PEST", "pkl_all_filename")
csv_all_filename = config.get("1_PEST", "csv_all_filename")

In [4]:
# API parameters
params = {
    "fran": fran,
    "till": till,
    "groda": groda,
    #"skadegorare": "blad",
}

# Authenticate and fetch data from the API
response = requests.get(API_URL, params=params, auth=(USERNAME, PASSWORD))

# Check for successful response
if response.status_code == 200:
    print("Data fetched successfully!")
    json_data = response.json()
else:
    print(f"Failed to fetch data: {response.status_code}")
    exit()

# Flatten the nested JSON into a structure suitable for creating a DataFrame
rows = []
for entry in json_data:
    for grading in entry.get('graderingstillfalleList', []):
        row = {
            "delomrade": entry.get("delomrade"),
            "ekologisk": entry.get("ekologisk"),
            "forforfrukt": entry.get("forforfrukt"),
            "forfrukt": entry.get("forfrukt"),
            "groda": entry.get("groda"),
            "jordart": entry.get("jordart"),
            "lan": entry.get("lan"),
            "latitud": entry.get("latitud"),
            "longitud": entry.get("longitud"),
            "plojt": entry.get("plojt"),
            "sadatum": entry.get("sadatum"),
            "skordear": entry.get("skordear"),
            "sort": entry.get("sort"),
            "graderingsdatum": grading.get("graderingsdatum"),
            "graderingstyp": grading.get("graderingstyp"),
            "utvecklingsstadium": grading.get("utvecklingsstadium"),
        }
        
        # Add "graderingList" data
        #for g in grading.get('graderingList', []):
        #    row[g['skadegorare']] = g['varde']  # Adding the varde values as separate columns for each pest
        #rows.append(row)

        for g in grading.get('graderingList', []):
            # Handle missing skadegorare and varde
            pest_name = g.get('skadegorare', 'Unknown')  # Default to 'Unknown' if skadegorare is missing
            value = g.get('varde', -1)  # Default to 0 if varde is missing
            if isinstance(value, str) and value.strip() == "":  # Check for empty strings or spaces
                value = -2  # Substitute empty strings with 0
    
            # Add the value to the column for this pest
            row[pest_name] = value
    
        rows.append(row)

# Convert the flattened data into a pandas DataFrame
df_0 = pd.DataFrame(rows)

print(df_0.shape)

Data fetched successfully!
(38157, 41)


In [5]:
# List of columns to select
columns_to_select = [
    'delomrade', 'lan', 'latitud', 'longitud', 'groda', 'sort', 'jordart', 
    'sadatum', 'skordear', 'graderingsdatum', 'graderingstyp',
    'forforfrukt', 'forfrukt', 'ekologisk', 'plojt', 
    'utvecklingsstadium', 'Bladfläcksvampar', 'Gulrost', 'Svartpricksjuka', 'Sädesbladlus'
]

# Replace spaces or empty strings in 'varde' with 0
#df.replace(to_replace=r'^\s*$', value=-3, regex=True, inplace=True)
# Replace all NaN values with 'Null'
#df.fillna('-1', inplace=True)

# Filter rows where delomrade is 'Östra Östergötland' and select the required columns
#df_pest = df_0[df_0['delomrade'] == 'Östra Östergötland'][columns_to_select]

# Filter rows based on below and select the required columns
df_pest_all = df_0[
    #(df_0['delomrade'] == 'Östra Östergötland') &
    (df_0['groda'] == 'Höstvete') &
    (df_0['graderingstyp'] == 'Veckovis') &
    (df_0['latitud'].str.strip() != '') &
    (df_0['longitud'].str.strip() != '')
][columns_to_select]

# Display the resulting DataFrame
print(df_pest_all.shape)

(25420, 20)


In [6]:
df_pest_all = df_pest_all.sort_values(by=['latitud', 'longitud', 'graderingsdatum'], ascending=True)

In [7]:
df_pest_all

,delomrade,lan,latitud,longitud,groda,sort,jordart,sadatum,skordear,graderingsdatum,graderingstyp,forforfrukt,forfrukt,ekologisk,plojt,utvecklingsstadium,Bladfläcksvampar,Gulrost,Svartpricksjuka,Sädesbladlus
4916,Gotland,Gotlands län,2441000,000,Höstvete,Julius,Lättlera (15-25 % ler),2016-09-18,2017,2017-05-01,Veckovis,Vårkorn,Vårkorn,False,True,26,0.0,0.0,0.0,0.0
4917,Gotland,Gotlands län,2441000,000,Höstvete,Julius,Lättlera (15-25 % ler),2016-09-18,2017,2017-05-08,Veckovis,Vårkorn,Vårkorn,False,True,30,0.0,0.0,0.0,0.0
4918,Gotland,Gotlands län,2441000,000,Höstvete,Julius,Lättlera (15-25 % ler),2016-09-18,2017,2017-05-15,Veckovis,Vårkorn,Vårkorn,False,True,32,0.0,0.0,0.0,0.0
4919,Gotland,Gotlands län,2441000,000,Höstvete,Julius,Lättlera (15-25 % ler),2016-09-18,2017,2017-05-22,Veckovis,Vårkorn,Vårkorn,False,True,33,0.0,0.0,0.0,0.0
4920,Gotland,Gotlands län,2441000,000,Höstvete,Julius,Lättlera (15-25 % ler),2016-09-18,2017,2017-05-29,Veckovis,Vårkorn,Vårkorn,False,True,37,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37153,Dalarna,Dalarnas län,None,None,Höstvete,Julius,None,None,2024,2024-05-13,Veckovis,None,None,False,None,30,6.0,0.0,6.0,NaN
37174,Gotland,Gotlands län,None,None,Höstvete,Kask,None,2023-09-15,2024,2024-05-13,Veckovis,Höstvete,Vårkorn,False,None,32,0.0,0.0,12.0,0.0
37887,Bohuslän,Västra Götalands län,None,None,Höstvete,Brons,Mellanlera (25-40 % ler),2023-09-17,2024,2024-05-13,Veckovis,Vårkorn,Höstvete,False,None,30,16.0,0.0,16.0,0.0
37986,Älvsborg,Västra Götalands län,None,None,Höstvete,Kask,None,None,2024,2024-05-13,Veckovis,None,None,False,None,32,8.0,0.0,8.0,0.0


In [8]:
# Get rows only for n distinct coordinates:
distinct_coords = df_pest_all[['latitud', 'longitud']].drop_duplicates()
print('# of all distinct coordinates: ', distinct_coords.shape)

# Remove rows where latitud or longitud contain only spaces or are empty
distinct_coords = distinct_coords.replace(r'^\s*$', pd.NA, regex=True).dropna().head(n_pest_coordinates)   # filters for n distinct coordinates only

df_pest = df_pest_all.merge(distinct_coords, on=['latitud', 'longitud'])

print(distinct_coords.shape)
print(df_pest.shape)

# of all distinct coordinates:  (884, 2)
(100, 2)
(5078, 20)


In [9]:
distinct_coords

,latitud,longitud
4916,2441000,000
6513,6135000,397000
1615,6136000,388000
1629,6136000,396000
34608,6138000,391000
...,...,...
1292,6199000,386000
1472,6200000,438000
34310,6201000,377000
6367,6201000,436000


In [10]:
df_pest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5078 entries, 0 to 5077
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   delomrade           5078 non-null   object 
 1   lan                 5078 non-null   object 
 2   latitud             5078 non-null   object 
 3   longitud            5078 non-null   object 
 4   groda               5078 non-null   object 
 5   sort                5078 non-null   object 
 6   jordart             1600 non-null   object 
 7   sadatum             2825 non-null   object 
 8   skordear            5078 non-null   int64  
 9   graderingsdatum     5078 non-null   object 
 10  graderingstyp       5078 non-null   object 
 11  forforfrukt         1495 non-null   object 
 12  forfrukt            3479 non-null   object 
 13  ekologisk           5078 non-null   bool   
 14  plojt               1600 non-null   object 
 15  utvecklingsstadium  5078 non-null   int64  
 16  Bladfl

In [11]:
# pickle - pandas dataframe
df_pest.to_pickle(pkl_filename)

# Save the DataFrame to a CSV file with UTF-8 encoding
df_pest.to_csv(csv_filename, index=False, encoding='utf-8-sig')  # Ensure proper encoding for special characters
print(f"Data saved to {csv_filename}")

# pickle - pandas dataframe
df_pest_all.to_pickle(pkl_all_filename)

# Save the DataFrame to a CSV file with UTF-8 encoding
df_pest_all.to_csv(csv_all_filename, index=False, encoding='utf-8-sig')  # Ensure proper encoding for special characters
print(f"Data saved to {csv_all_filename}")

Data saved to df_1_pest_t1.csv
Data saved to df_1_pest_all_t1.csv
